In [ ]:
import sys
import warnings

sys.path.append("../../")
warnings.filterwarnings("ignore")

# set working directory to root 
import os
os.chdir("../../")
print(os.getcwd())
root = os.getcwd()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from einops import rearrange
from matplotlib.colors import ListedColormap

from src.models.datamodule import MineDataModule
from src.models.cnn.model import MineSegmentorCNN

from src.visualization.visualization_funcs import plot_pred_vs_true_mask

In [ ]:
# for development
%load_ext autoreload
%autoreload 2

### Define paths and parameters

In [ ]:
# 1024 model
# MINESEG_CHECKPOINT_PATH = ("models/cnn/mineseg-cnn_epoch-07_val-iou-0.5315.ckpt")
# CHIP_SIZE = 1024
# TRAIN_CHIP_DIR = "data/processed/chips/npy/1024/train/chips/"
# TRAIN_LABEL_DIR = " data/processed/chips/npy/1024/train/labels/"
# VAL_CHIP_DIR = "data/processed/chips/npy/1024/val/chips/"
# VAL_LABEL_DIR = "data/processed/chips/npy/1024/val/labels/"
# TEST_CHIP_DIR = "data/processed/chips/npy/1024/test/chips/"
# TEST_LABEL_DIR = "data/processed/chips/npy/1024/test/labels/"
# CHIP_SIZE = 1024

# 512 model
MINESEG_CHECKPOINT_PATH = ("models/cnn/mineseg-cnn_epoch-05_val-iou-0.5553.ckpt")
CHIP_SIZE = 512
TRAIN_CHIP_DIR = "data/processed/chips/npy/train/chips/"
TRAIN_LABEL_DIR = "data/processed/chips/npy/train/labels/"
VAL_CHIP_DIR = "data/processed/chips/npy/val/chips/"
VAL_LABEL_DIR = "data/processed/chips/npy/val/labels/"
TEST_CHIP_DIR = "data/processed/chips/npy/test/chips/"
TEST_LABEL_DIR = "data/processed/chips/npy/test/labels/"
FILTERED_DS = "/data/processed/mining_tiles_with_masks_and_bounding_boxes_filtered.gpkg"


METADATA_PATH = "configs/cnn/cnn_segment_metadata.yaml"
BATCH_SIZE = 1
NUM_WORKERS = 4
PLATFORM = "sentinel-2-l2a"

In [ ]:
is_lightning = True
if is_lightning:
    root = root + "/workspaces/mine-segmentation"
    MINESEG_CHECKPOINT_PATH = root + "/" + MINESEG_CHECKPOINT_PATH
    METADATA_PATH = root + "/" + METADATA_PATH
    TRAIN_CHIP_DIR = root +  "/" +TRAIN_CHIP_DIR
    TRAIN_LABEL_DIR = root + "/" + TRAIN_LABEL_DIR
    VAL_CHIP_DIR = root + "/" + VAL_CHIP_DIR
    VAL_LABEL_DIR = root + "/" + VAL_LABEL_DIR
    TEST_CHIP_DIR = root + "/" + TEST_CHIP_DIR
    TEST_LABEL_DIR = root + "/" + TEST_LABEL_DIR
    FILTERED_DS = root + "/" + FILTERED_DS
    print(MINESEG_CHECKPOINT_PATH)

### Model Loading

In [ ]:
def get_model(checkpoint_path: str) -> MineSegmentorCNN:
    # check if gpu is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    map_location=torch.device(device)
    checkpoint = torch.load(checkpoint_path, map_location=map_location)
    model_config = checkpoint["hyper_parameters"]
    model = MineSegmentorCNN.load_from_checkpoint(checkpoint_path, **model_config)
    model.eval()
    return model

### Data Preparation

In [ ]:
def get_data(
    train_chip_dir,
    train_label_dir,
    val_chip_dir,
    val_label_dir,
    test_chip_dir,
    test_label_dir,
    metadata_path,
    batch_size,
    num_workers,
    platform,
    data_augmentation,
    index=None
):
    dm = MineDataModule(
        train_chip_dir=train_chip_dir,
        train_label_dir=train_label_dir,
        val_chip_dir=val_chip_dir,
        val_label_dir=val_label_dir,
        test_chip_dir=test_chip_dir,
        test_label_dir=test_label_dir,
        metadata_path=metadata_path,
        batch_size=batch_size,
        num_workers=num_workers,
        platform=platform,
        data_augmentation=data_augmentation,
    )
    
    dm.setup(stage="test")
    
    if index is not None:
        test_dl = iter(dm.test_dataloader())
        for i in range(index + 1):
            batch = next(test_dl)
        metadata = dm.metadata
        return batch, metadata
    else:
        test_dl = dm.test_dataloader()
        batch = next(iter(test_dl))
        metadata = dm.metadata
        return batch, metadata
    metadata = dm.metadata
    return batch, metadata

### Prediction

In [ ]:
def run_prediction(model, batch):
    with torch.no_grad():
        image = batch["pixels"]
        outputs = model(image)
    outputs = F.interpolate(
        outputs, size=(CHIP_SIZE, CHIP_SIZE), mode="bilinear", align_corners=False
    )
    return outputs

### Post-Processing

In [ ]:
def post_process(batch, outputs, metadata):
    prob_mask = outputs.cpu().numpy()
    pred_mask = (prob_mask > 0.5).astype(float)
    labels = batch["label"].detach().cpu().numpy()
    pixels = batch["pixels"].detach().cpu().numpy()

    # normalize and clip the image ranges
    pixels = (pixels - pixels.min()) / (pixels.max() - pixels.min())
    pixels = np.clip(pixels, 0, 1)

    images = pixels[0]
    labels = labels[0]
    prob_mask = prob_mask[0]
    pred_mask = pred_mask[0].astype(float)

    images = images.transpose((1,2,0))
    prob_mask = prob_mask.transpose((1,2,0))
    pred_mask = pred_mask.transpose((1,2,0)).astype(float)

    return images, labels, prob_mask, pred_mask

### Plotting

In [ ]:
def plot_predictions(images, labels, probas, preds):
    fig, axes = plt.subplots(1, 5, figsize=(15, 6))

    # Plot the image
    axes[0].imshow(images)
    axes[0].axis("off")
    axes[0].set_title("Image", fontsize=12)

    # Plot the actual segmentation
    axes[1].imshow(labels, vmin=0, vmax=1)
    axes[1].axis("off")
    axes[1].set_title("Actual", fontsize=12)

    # Plot the predicted segmentation
    axes[2].imshow(preds, vmin=0, vmax=1)
    axes[2].axis("off")
    axes[2].set_title("Pred", fontsize=12)

    # Plot the predicted segmentation
    axes[3].imshow(probas, vmin=0, vmax=1)
    axes[3].axis("off")
    axes[3].set_title("Proba", fontsize=12)

    # Plot the plot_pred_vs_true_mask
    plot_pred_vs_true_mask(images, labels, preds.squeeze(), ax=axes[4], add_legend=False)
    axes[4].set_title("Pred vs True", fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
# Load model
model = get_model(MINESEG_CHECKPOINT_PATH)

In [ ]:
import random

nr_test_chips = len(os.listdir(TEST_CHIP_DIR))
sample = random.sample(range(nr_test_chips), 7)

for index in sample:

    # Get data
    batch, metadata = get_data(
        TRAIN_CHIP_DIR,
        TRAIN_LABEL_DIR,
        VAL_CHIP_DIR,
        VAL_LABEL_DIR,
        TEST_CHIP_DIR,
        TEST_LABEL_DIR,
        METADATA_PATH,
        BATCH_SIZE,
        NUM_WORKERS,
        PLATFORM,
        data_augmentation=False,
        index=index
    )

    # Move batch to GPU
    if torch.cuda.is_available():
        batch = {k: v.to("cuda") for k, v in batch.items()}

    # Run prediction
    outputs = run_prediction(model, batch)
    
    # Post-process the results
    images, labels, probas, preds = post_process(batch, outputs, metadata)

    # Plot the predictions
    plot_predictions(images, labels, probas, preds)


In [ ]:
fig = plot_pred_vs_true_mask(images, labels, preds.squeeze(), ax=None)

### Check Data Transformations

In [ ]:
import matplotlib.pyplot as plt

datamodule = MineDataModule(
    train_chip_dir=TRAIN_CHIP_DIR,
    train_label_dir=TRAIN_LABEL_DIR,
    val_chip_dir=VAL_CHIP_DIR,
    val_label_dir=VAL_LABEL_DIR,
    test_chip_dir=TEST_CHIP_DIR,
    test_label_dir=TEST_LABEL_DIR,
    metadata_path=METADATA_PATH,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    platform=PLATFORM,
    data_augmentation=True,
)

datamodule.setup(stage="fit")

In [ ]:
batch = next(iter(datamodule.train_dataloader()))

batch["pixels"].shape, batch["label"].shape

# squeeze the batch size dimension
pixels = batch["pixels"].squeeze(0)
label = batch["label"].squeeze(0)

# rearrange for plotting
pixels = rearrange(pixels, "c h w -> h w c")
label = rearrange(label, "h w -> h w")

# normalize
pixels = (pixels - pixels.min()) / (pixels.max() - pixels.min())

# plot the image and label side by side
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# plot the image
axes[0].imshow(pixels)
axes[0].axis("off")
axes[0].set_title("Image", fontsize=12)

# plot the label
axes[1].imshow(label, cmap="viridis")
axes[1].axis("off")
axes[1].set_title("Label", fontsize=12)

plt.tight_layout()
plt.show()

### Run inference and metrics on test dataset

In [ ]:
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, roc_auc_score
import pandas as pd
from tqdm import tqdm
import numpy as np

# list all files in the test chip directory
test_chips = os.listdir(TEST_CHIP_DIR)

# Initialize a DataFrame to store the results
results = pd.DataFrame(columns=['image', "file_name", 'iou', 'f1', 'accuracy', 'recall', 'precision'])

# Define batch size
BATCH_SIZE = 64

# Calculate number of batches
num_batches = int(np.ceil(len(test_chips) / BATCH_SIZE))

# Move model to GPU
if torch.cuda.is_available():
    model = model.to("cuda")

for i in tqdm(range(num_batches)):

    file_names = test_chips[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]

    # Get data
    batch, metadata = get_data(
        TRAIN_CHIP_DIR,
        TRAIN_LABEL_DIR,
        VAL_CHIP_DIR,
        VAL_LABEL_DIR,
        TEST_CHIP_DIR,
        TEST_LABEL_DIR,
        METADATA_PATH,
        BATCH_SIZE,
        NUM_WORKERS,
        PLATFORM,
        data_augmentation=False,
        index=i  # Get the next batch of images
    )

    # Move batch to GPU
    if torch.cuda.is_available():
        batch = {k: v.to("cuda") for k, v in batch.items()}

    # Run prediction
    outputs = run_prediction(model, batch)

    labels = batch["label"].detach().cpu().numpy()
    preds = outputs.detach().cpu().numpy()
    preds = (preds > 0.5).astype(float)

    # Calculate the metrics for each image in the batch
    for j in range(len(outputs)):
        iou = jaccard_score(labels[j].flatten(), preds[j].flatten())
        f1 = f1_score(labels[j].flatten(), preds[j].flatten())
        accuracy = accuracy_score(labels[j].flatten(), preds[j].flatten())
        recall = recall_score(labels[j].flatten(), preds[j].flatten())
        precision = precision_score(labels[j].flatten(), preds[j].flatten())
        # roc_auc = roc_auc_score(labels[j].flatten(), preds[j].flatten())

        # Add the results to the DataFrame
        results = pd.concat(
            [results, 
            pd.DataFrame(
                {
                    'image': [i * BATCH_SIZE + j], 
                    'file_name': [file_names[j]],
                    'iou': [iou], 
                    'f1': [f1], 
                    'accuracy': [accuracy], 
                    'recall': [recall], 
                    'precision': [precision], 
                    # 'roc_auc': [roc_auc]
                })])

In [ ]:
import geopandas as gpd
import datetime
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H")

# load the processed dataset
tiles = gpd.read_file(FILTERED_DS, layer="tiles")
test_tiles = tiles[tiles["split"] == "test"]

# extract the tile_id from the file_name
results["tile_id"] = results["file_name"].apply(lambda x: x.split("_")[0])

# save as csv in the reports folder
output_path = root + f"/reports/cnn_testset_metrics_per_chip_{current_datetime}.csv"
results.to_csv(output_path, index=False)

# Select only the numeric columns
numeric_columns = results.select_dtypes(include=['float64', 'int64'])

# Add the 'tile_id' column to the numeric columns
numeric_columns['tile_id'] = results['tile_id']

# Group by 'tile_id' and calculate the mean of the numeric columns
results_grouped = numeric_columns.groupby('tile_id').mean().reset_index()

# Merge the results with the test_tiles
test_tiles = test_tiles.merge(results_grouped, on='tile_id')

# save as csv in the reports folder
output_path = root + f"/reports/cnn_testset_metrics_aggregated_{current_datetime}.csv"
test_tiles.to_csv(output_path, index=False)

# calculate overall metrics
iou = results_grouped["iou"].mean()
f1 = results_grouped["f1"].mean()
accuracy = results_grouped["accuracy"].mean()
recall = results_grouped["recall"].mean()
precision = results_grouped["precision"].mean()

# print the results
print(f"Mean IoU: {iou}")
print(f"Mean F1: {f1}")
print(f"Mean Accuracy: {accuracy}")
print(f"Mean Recall: {recall}")
print(f"Mean Precision: {precision}")


In [ ]:
test_tiles.head()

In [ ]:
test_tiles.groupby("minetype1")[["iou", "f1", "accuracy", "recall", "precision"]].mean()

In [ ]:
test_tiles.groupby("minetype2")[["iou", "f1", "accuracy", "recall", "precision"]].mean()

In [ ]:
test_tiles.groupby("preferred_dataset")[["iou", "f1", "accuracy", "recall", "precision"]].mean()

In [ ]:
# print a histogram of the iou scores
test_tiles["iou"].hist(bins=20)